In [2]:
import numpy as np
import pandas as pd
import h5py
import pysam
import os
from modisco.visualization import viz_sequence
from modisco import util
from matplotlib import pyplot as plt
import pybedtools

pd.options.display.max_rows = 500
pd.options.display.max_columns = 500

In [13]:
#dttype = 'atac'
#mode = 'counts'

#modisco_path = '/oak/stanford/groups/akundaje/projects/chrombpnet_paper_new/ATAC_PE/K562/uncorrected_model_08.22.2022_filters_512_dil_8/SIGNAL/modisco_crop_500/modisco_results_allChroms_'+mode+'.hdf5'
#ppm_dir = '/mnt/lab_data3/anusri/chrombpnet/results/chrombpnet/ATAC_PE/K562/uncorrected_model_08.22.2022_filters_512_dil_8/'
#tomtom = pd.read_csv("/oak/stanford/groups/akundaje/projects/chrombpnet_paper_new/ATAC_PE/K562/uncorrected_model_08.22.2022_filters_512_dil_8/SIGNAL/modisco_crop_500/" + mode + ".tomtom.tsv", sep="\t")

dttype = 'dnase'
mode = "profile"

modisco_path = '/oak/stanford/groups/akundaje/projects/chrombpnet_paper_new/DNASE_PE/K562/uncorrected_model_08.31.2022_filters_512_dil_8/SIGNAL/modisco_crop_500/modisco_results_allChroms_'+mode+'.hdf5'
ppm_dir = '/mnt/lab_data3/anusri/chrombpnet/results/chrombpnet/DNASE_PE/K562/uncorrected_model_08.31.2022_filters_512_dil_8/'
tomtom = pd.read_csv("/oak/stanford/groups/akundaje/projects/chrombpnet_paper_new/DNASE_PE/K562/uncorrected_model_08.31.2022_filters_512_dil_8/SIGNAL/modisco_crop_500/" + mode + ".tomtom.tsv", sep="\t")




background=[0.25, 0.25, 0.25, 0.25]

In [14]:
def trim_motif_new(cwm, motif, trim_threshold=0.20):
    """
    Given the PFM and motif (both L x 4 arrays) (the motif could be the
    PFM itself), trims `motif` by cutting off flanks of low information
    content in `pfm`. `min_ic` is the minimum required information
    content. If specified this trimmed motif will be extended on either
    side by `pad` bases.
    If no base passes the `min_ic` threshold, then no trimming is done.
    """
    
    score = np.sum(np.abs(cwm), axis=1)
    trim_thresh = np.max(score) * trim_threshold  # Cut off anything less than 30% of max score
    pass_inds = np.where(score >= trim_thresh)[0]
    trimmed = motif[np.min(pass_inds): np.max(pass_inds) + 1]
 
    if not trimmed.size:
        return motif
    
    return trimmed

def import_tfmodisco_motifs(tfm_results_path, trim=True, only_pos=True):
    """
    Imports the PFMs to into a dictionary, mapping `(x, y)` to the PFM,
    where `x` is the metacluster index and `y` is the pattern index.
    Arguments:
        `tfm_results_path`: path to HDF5 containing TF-MoDISco results
        `out_dir`: where to save motifs
        `trim`: if True, trim the motif flanks based on information content
        `only_pos`: if True, only return motifs with positive contributions
    Returns the dictionary of PFMs.
    """ 
    pfms = {}
    with h5py.File(tfm_results_path, "r") as f:
        metaclusters = f["metacluster_idx_to_submetacluster_results"]
        num_metaclusters = len(metaclusters.keys())
        for metacluster_i, metacluster_key in enumerate(metaclusters.keys()):
            metacluster = metaclusters[metacluster_key]
            if "patterns" not in metacluster["seqlets_to_patterns_result"]:
                continue
            patterns = metacluster["seqlets_to_patterns_result"]["patterns"]
            num_patterns = len(patterns["all_pattern_names"][:])
            for pattern_i, pattern_name in enumerate(patterns["all_pattern_names"][:]):
                pattern_name = pattern_name.decode()
                pattern = patterns[pattern_name]
                pfm = pattern["sequence"]["fwd"][:]
                cwm = pattern["task0_contrib_scores"]["fwd"][:]
                
                # Check that the contribution scores are overall positive
                if only_pos and np.sum(cwm) < 0:
                    continue
                    
                if trim:
                    pfm = trim_motif_new(cwm, cwm)
                    
                pfms["%d_%d" % (metacluster_i,pattern_i)] = pfm
    return pfms

In [15]:
pfms = import_tfmodisco_motifs(modisco_path)

In [16]:
for key in pfms:
    f = open(os.path.join(ppm_dir,mode+"_"+key+".pfm"),"w")
    #print(pfms[key])
    np.savetxt(f, pfms[key], fmt='%f')
    f.close()
    

In [17]:
tomtom.head()

,Pattern,Num_Seqlets,Match_1,q-value,Match_2,q-value.1,Match_3,q-value.2,Match_4,q-value.3,Match_5,q-value.4,Match_6,q-value.5,Match_7,q-value.6,Match_8,q-value.7,Match_9,q-value.8,Match_10,q-value.9
0,metacluster_0.pattern_0,6868,CTCF_MA0139.1,1.048660e-15,CTCF_HUMAN.H11MO.0.A,3.036260e-12,CTCF_MOUSE.H11MO.0.A,3.532730e-11,CTCFL_HUMAN.H11MO.0.A,2.675630e-07,CTCFL_MOUSE.H11MO.0.A,5.714800e-07,CTCF_C2H2_1,7.527910e-07,CTCFL_MA1102.1,0.000010,ZIC2_MOUSE.H11MO.0.C,0.177648,ZIC3_HUMAN.H11MO.0.B,0.177648,ZIC3_MOUSE.H11MO.0.A,0.177648
1,metacluster_0.pattern_1,6333,GATA4_HUMAN.H11MO.0.A,1.622640e-02,GATA4_MOUSE.H11MO.0.A,1.622640e-02,GATA6_MOUSE.H11MO.0.A,1.622640e-02,GATA2_MA0036.3,1.622640e-02,Gata4_MA0482.1,1.622640e-02,GATA2_MOUSE.H11MO.0.A,1.622640e-02,GATA6_HUMAN.H11MO.0.A,0.016226,GATA6_MA1104.1,0.016226,GATA1_HUMAN.H11MO.0.A,0.018723,GATA1_MOUSE.H11MO.0.A,0.018723
2,metacluster_0.pattern_2,4999,SP1_MA0079.3,2.663700e-07,KLF5_MA0599.1,2.865510e-07,SP3_HUMAN.H11MO.0.B,2.066330e-06,SP3_MOUSE.H11MO.0.B,2.066330e-06,SP1_MOUSE.H11MO.0.A,3.875050e-06,SP1_HUMAN.H11MO.0.A,4.480970e-06,SP1_C2H2_1,0.000006,SP2_MA0516.1,0.000020,SP4_HUMAN.H11MO.0.A,0.000020,SP4_MOUSE.H11MO.0.B,0.000020
3,metacluster_0.pattern_3,3270,FOSB_HUMAN.H11MO.0.A,3.890440e-05,FOSB_MOUSE.H11MO.0.A,3.890440e-05,JUN_MOUSE.H11MO.0.A,3.890440e-05,BACH2_HUMAN.H11MO.0.A,3.890440e-05,BACH2_MOUSE.H11MO.0.A,3.890440e-05,FOSL2_MOUSE.H11MO.0.A,3.890440e-05,JUND_HUMAN.H11MO.0.A,0.000039,JUND_MOUSE.H11MO.0.A,0.000039,FOSL1_HUMAN.H11MO.0.A,0.000039,FOSL1_MOUSE.H11MO.0.A,0.000039
4,metacluster_0.pattern_4,1590,TGIF1_HUMAN.H11MO.0.A,2.293890e-01,MAFB_HUMAN.H11MO.0.B,2.936690e-01,MAFB_MOUSE.H11MO.0.B,2.936690e-01,TGIF1_MOUSE.H11MO.0.A,2.936690e-01,REST_HUMAN.H11MO.0.A,2.936690e-01,CLOCK_HUMAN.H11MO.0.C,2.936690e-01,CLOCK_MOUSE.H11MO.0.C,0.293669,TFE3_MA0831.2,0.304428,REST_MOUSE.H11MO.0.A,0.304428,Arntl_MA0603.1,0.304428


In [18]:

tomtom["Pattern"] = tomtom["Pattern"].str.replace("metacluster_","").str.replace(".pattern_","_")

/users/anusri/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [19]:
tomtom.head()

,Pattern,Num_Seqlets,Match_1,q-value,Match_2,q-value.1,Match_3,q-value.2,Match_4,q-value.3,Match_5,q-value.4,Match_6,q-value.5,Match_7,q-value.6,Match_8,q-value.7,Match_9,q-value.8,Match_10,q-value.9
0,0_0,6868,CTCF_MA0139.1,1.048660e-15,CTCF_HUMAN.H11MO.0.A,3.036260e-12,CTCF_MOUSE.H11MO.0.A,3.532730e-11,CTCFL_HUMAN.H11MO.0.A,2.675630e-07,CTCFL_MOUSE.H11MO.0.A,5.714800e-07,CTCF_C2H2_1,7.527910e-07,CTCFL_MA1102.1,0.000010,ZIC2_MOUSE.H11MO.0.C,0.177648,ZIC3_HUMAN.H11MO.0.B,0.177648,ZIC3_MOUSE.H11MO.0.A,0.177648
1,0_1,6333,GATA4_HUMAN.H11MO.0.A,1.622640e-02,GATA4_MOUSE.H11MO.0.A,1.622640e-02,GATA6_MOUSE.H11MO.0.A,1.622640e-02,GATA2_MA0036.3,1.622640e-02,Gata4_MA0482.1,1.622640e-02,GATA2_MOUSE.H11MO.0.A,1.622640e-02,GATA6_HUMAN.H11MO.0.A,0.016226,GATA6_MA1104.1,0.016226,GATA1_HUMAN.H11MO.0.A,0.018723,GATA1_MOUSE.H11MO.0.A,0.018723
2,0_2,4999,SP1_MA0079.3,2.663700e-07,KLF5_MA0599.1,2.865510e-07,SP3_HUMAN.H11MO.0.B,2.066330e-06,SP3_MOUSE.H11MO.0.B,2.066330e-06,SP1_MOUSE.H11MO.0.A,3.875050e-06,SP1_HUMAN.H11MO.0.A,4.480970e-06,SP1_C2H2_1,0.000006,SP2_MA0516.1,0.000020,SP4_HUMAN.H11MO.0.A,0.000020,SP4_MOUSE.H11MO.0.B,0.000020
3,0_3,3270,FOSB_HUMAN.H11MO.0.A,3.890440e-05,FOSB_MOUSE.H11MO.0.A,3.890440e-05,JUN_MOUSE.H11MO.0.A,3.890440e-05,BACH2_HUMAN.H11MO.0.A,3.890440e-05,BACH2_MOUSE.H11MO.0.A,3.890440e-05,FOSL2_MOUSE.H11MO.0.A,3.890440e-05,JUND_HUMAN.H11MO.0.A,0.000039,JUND_MOUSE.H11MO.0.A,0.000039,FOSL1_HUMAN.H11MO.0.A,0.000039,FOSL1_MOUSE.H11MO.0.A,0.000039
4,0_4,1590,TGIF1_HUMAN.H11MO.0.A,2.293890e-01,MAFB_HUMAN.H11MO.0.B,2.936690e-01,MAFB_MOUSE.H11MO.0.B,2.936690e-01,TGIF1_MOUSE.H11MO.0.A,2.936690e-01,REST_HUMAN.H11MO.0.A,2.936690e-01,CLOCK_HUMAN.H11MO.0.C,2.936690e-01,CLOCK_MOUSE.H11MO.0.C,0.293669,TFE3_MA0831.2,0.304428,REST_MOUSE.H11MO.0.A,0.304428,Arntl_MA0603.1,0.304428


In [20]:
tomtom[["Pattern","Num_Seqlets"]].to_csv(os.path.join(ppm_dir,mode+"_counts.csv"),sep=",",index=False, header=False)